In [1]:
import json
import time
import re
import os
import random
from typing import Dict, List, Union, Tuple

import requests
from bs4 import BeautifulSoup, Tag
from tqdm.notebook import tqdm

In [2]:
def get_soup(url: str, attempts:int = 10) -> BeautifulSoup:
    """request a given url and return BeautifulSoup object"""
    try:
        headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36"}
        res = requests.get(url, headers=headers)
    except:
        raise
    soup = BeautifulSoup(res.content)
    if len(soup.find_all("div", {"class" : re.compile('SongInfo__Credit*')})) == 0 and attempts > 0:
        time.sleep(random.random())
        soup = get_soup(url, attempts-1)
    return soup

In [3]:
def get_songs_url_from_album_page(album_url: str) -> List[str]:
    """given an album page, returns all song urls"""
    soup = get_soup(album_url)
    return [title.parent["href"] for title in soup.find_all(class_="chart_row-content-title")]

In [4]:
def get_lyrics_text(song_soup):
    """extracts the lyrics from a song page"""
    text = []
    lyrics_div = song_soup.select("div[class=lyrics]")
    if not len(lyrics_div):
        lyrics_div = song_soup.find_all("div", {"class": re.compile("Lyrics__Container*")})
        if not len(lyrics_div):
            lyrics_div = song_soup.find_all("div", {"class": re.compile("LyricsPlaceholder__Message*")})[0]
            if "instrumental" in lyrics_div.text:
                return Tag(name="div")
        for ld in lyrics_div:
            for br in ld.find_all("br"):
                br.replace_with("\n")
    for ld in lyrics_div:
        text.extend(ld.text.strip().split("\n"))
    return text

In [5]:
def get_song_lyrics(song_soup) -> str:
    lyrics = get_lyrics_text(song_soup)
    lyrics = list(filter(lambda line: len(line) > 0, lyrics))
    return lyrics

In [6]:
def get_song_title(song_soup):
    title = song_soup.find_all("h1", {"class" : re.compile('SongHeader__Title*')})
    if len(title):
        pass
    else:
        title = song_soup.find_all("h1", {"class" : re.compile('header_with_cover_art-primary_info-title')})
    title = title[0].text
    return title

In [7]:
def get_album_title(song_soup):
    album_title = song_soup.find_all("div", {"class": re.compile("HeaderTracklist__Album-*")})
    if len(album_title):
        album_title = album_title[-1].text
    else:
        album_title = song_soup.find_all("a", {"class": re.compile("song_album-info-title")})[0].text.strip()
    return album_title

In [8]:
def get_song_metadata(song_soup):
    metadata = {}
    metadata["title"] = get_song_title(song_soup)    
    metadata["album_title"] = get_album_title(song_soup)
    for song_credit in song_soup.find_all("div", {"class" : re.compile('SongInfo__Credit*')}):
        values = []
        for i, x in enumerate(song_credit):
            if isinstance(x, Tag):
                if i == 0:
                    label = x.text
                else:
                    values.append(x.text)
        metadata[label] = values
    return metadata

In [9]:
def get_song_data_form_song_url(song_url: str) -> Dict[str, Union[str, Dict]]:
    print(song_url)
    soup = get_soup(song_url)
    lyrics = get_song_lyrics(soup)
    metadata = get_song_metadata(soup)
    return {
        "lyrics": lyrics,
        "metadata": metadata
    }

In [10]:
album_urls = [
    "https://genius.com/albums/Mac-miller/K-i-d-s-deluxe",
    "https://genius.com/albums/Mac-miller/K-i-d-s-re-release",
    "https://genius.com/albums/Mac-miller/Circles-deluxe",
    "https://genius.com/albums/Mac-miller/Circles",
    "https://genius.com/albums/Mac-miller/Spotify-singles",
    "https://genius.com/albums/Mac-miller/Swimming",
    "https://genius.com/albums/Mac-miller/The-divine-feminine",
    "https://genius.com/albums/Mac-miller/Go-od-am",
    "https://genius.com/albums/Mac-miller/Faces",
    "https://genius.com/albums/Mac-miller/Live-from-space",
    "https://genius.com/albums/Mac-miller/Watching-movies-with-the-sound-off",
    "https://genius.com/albums/Mac-miller/Live-from-london",
    "https://genius.com/albums/Mac-miller/Macadelic",
    "https://genius.com/albums/Mac-miller/Blue-slide-park",
    "https://genius.com/albums/Mac-miller/I-love-life-thank-you",
    "https://genius.com/albums/Mac-miller/On-and-on-and-beyond",
    "https://genius.com/albums/Mac-miller/Best-day-ever",
    "https://genius.com/albums/Mac-miller/K-i-d-s",
    "https://genius.com/albums/Mac-miller/The-high-life",
    "https://genius.com/albums/Mac-miller/The-jukebox-prelude-to-class-clown",
    "https://genius.com/albums/Mac-miller/But-my-mackin-aint-easy-original-version",
    "https://genius.com/albums/Mac-miller/But-my-mackin-aint-easy",
    "https://genius.com/albums/Mac-miller/Balloonerism",
    "https://genius.com/albums/Mac-miller/Unreleased-songs",
]

songs404 = [
    "https://genius.com/Mac-miller-unreleased-songs-discography-list-annotated",
    "https://genius.com/Mac-miller-we-brothers-in-arms-lyrics",
    "https://genius.com/Mac-miller-too-green-scene-lyrics",
    "https://genius.com/Mac-miller-haters-last-meal-lyrics",
    "https://genius.com/Mac-miller-crooklyn-dodgers-lyrics",
    "https://genius.com/Mac-miller-how-we-do-lyrics",
    "https://genius.com/Mac-miller-the-world-is-mine-lyrics",
    "https://genius.com/Mac-miller-war-games-freestyle-lyrics",
    "https://genius.com/Mac-miller-mac-is-like-lyrics",
    "https://genius.com/Mac-miller-it-gets-no-better-than-this-lyrics",
    "https://genius.com/Mac-miller-coffee-shop-freestyle-lyrics"
]

In [11]:
data_dir = "../data"

In [12]:
data = []
processed_songs = []
for album_url in tqdm(album_urls):
    song_urls = get_songs_url_from_album_page(album_url)
    for song_url in tqdm(song_urls):
        if song_url in songs404 or song_url in processed_songs:
            continue
        processed_songs.append(song_url)
        song_data = get_song_data_form_song_url(song_url)
        song_data["url"] = song_url
        data.append(song_data)
        time.sleep(random.random())
    with open(os.path.join(data_dir, "raw_lyrics.json"), "w") as fh:
        json.dump(data, fh, ensure_ascii=False)

https://genius.com/Mac-miller-kickin-incredibly-dope-shit-intro-lyrics
https://genius.com/Mac-miller-outside-lyrics
https://genius.com/Mac-miller-get-em-up-lyrics
https://genius.com/Mac-miller-nikes-on-my-feet-lyrics
https://genius.com/Mac-miller-senior-skip-day-lyrics
https://genius.com/Mac-miller-the-spins-lyrics
https://genius.com/Mac-miller-dont-mind-if-i-do-lyrics
https://genius.com/Mac-miller-paper-route-lyrics
https://genius.com/Mac-miller-good-evening-lyrics
https://genius.com/Mac-miller-ride-around-lyrics
https://genius.com/Mac-miller-knock-knock-lyrics
https://genius.com/Mac-miller-mad-flava-heavy-flow-interlude-lyrics
https://genius.com/Mac-miller-kool-aid-and-frozen-pizza-lyrics
https://genius.com/Mac-miller-all-i-want-is-you-lyrics
https://genius.com/Mac-miller-poppy-lyrics
https://genius.com/Mac-miller-face-in-the-crowd-lyrics
https://genius.com/Mac-miller-ayye-lyrics
https://genius.com/Mac-miller-back-in-the-day-lyrics



https://genius.com/Mac-miller-circles-lyrics
https://genius.com/Mac-miller-complicated-lyrics
https://genius.com/Mac-miller-blue-world-lyrics
https://genius.com/Mac-miller-good-news-lyrics
https://genius.com/Mac-miller-i-can-see-lyrics
https://genius.com/Mac-miller-everybody-lyrics
https://genius.com/Mac-miller-woods-lyrics
https://genius.com/Mac-miller-hand-me-downs-lyrics
https://genius.com/Mac-miller-thats-on-me-lyrics
https://genius.com/Mac-miller-hands-lyrics
https://genius.com/Mac-miller-surf-lyrics
https://genius.com/Mac-miller-once-a-day-lyrics
https://genius.com/Mac-miller-right-lyrics
https://genius.com/Mac-miller-floating-lyrics



https://genius.com/Mac-miller-dunno-spotify-singles-lyrics
https://genius.com/Mac-miller-nothing-from-nothing-lyrics



https://genius.com/Mac-miller-come-back-to-earth-lyrics
https://genius.com/Mac-miller-hurt-feelings-lyrics
https://genius.com/Mac-miller-whats-the-use-lyrics
https://genius.com/Mac-miller-perfecto-lyrics
https://genius.com/Mac-miller-self-care-lyrics
https://genius.com/Mac-miller-wings-lyrics
https://genius.com/Mac-miller-ladders-lyrics
https://genius.com/Mac-miller-small-worlds-lyrics
https://genius.com/Mac-miller-conversation-pt-1-lyrics
https://genius.com/Mac-miller-dunno-lyrics
https://genius.com/Mac-miller-jet-fuel-lyrics
https://genius.com/Mac-miller-2009-lyrics
https://genius.com/Mac-miller-so-it-goes-lyrics



https://genius.com/Mac-miller-congratulations-lyrics
https://genius.com/Mac-miller-dang-lyrics
https://genius.com/Mac-miller-stay-lyrics
https://genius.com/Mac-miller-skin-lyrics
https://genius.com/Mac-miller-cinderella-lyrics
https://genius.com/Mac-miller-planet-god-damn-lyrics
https://genius.com/Mac-miller-soulmate-lyrics
https://genius.com/Mac-miller-we-lyrics
https://genius.com/Mac-miller-my-favorite-part-lyrics
https://genius.com/Mac-miller-god-is-fair-sexy-nasty-lyrics
https://genius.com/Mac-miller-dang-radio-edit-lyrics



...

https://genius.com/Mac-miller-intro-the-jukebox-annotated
https://genius.com/Mac-miller-sound-like-lyrics
https://genius.com/Mac-miller-barz-for-dayz-lyrics
https://genius.com/Mac-miller-pa-hustla-lyrics
https://genius.com/Mac-miller-what-up-cousin-lyrics
https://genius.com/Mac-miller-got-a-clue-lyrics
https://genius.com/Mac-miller-get-it-on-the-floor-lyrics
https://genius.com/Mac-miller-late-night-lyrics
https://genius.com/Mac-miller-cruisin-lyrics
https://genius.com/Mac-miller-swing-set-lyrics
https://genius.com/Mac-miller-love-my-name-lyrics
https://genius.com/Mac-miller-snap-back-lyrics
https://genius.com/Mac-miller-js-on-my-feet-lyrics
https://genius.com/Mac-miller-on-some-real-shit-100000-bars-lyrics
https://genius.com/Mac-miller-the-chow-line-lyrics
https://genius.com/Mac-miller-a-night-in-the-studio-lyrics
https://genius.com/Mac-miller-my-lady-lyrics
https://genius.com/Mac-miller-comin-soon-lyrics
https://genius.com/Mac-miller-keep-me-alive-lyrics
https://genius.com/Mac-miller-

https://genius.com/Mac-miller-mackin-aint-easy-lyrics
https://genius.com/Mac-miller-warhead-lyrics
https://genius.com/Mac-miller-hey-you-lyrics
https://genius.com/Mac-miller-dead-presidents-lyrics
https://genius.com/Mac-miller-good-days-are-gone-lyrics
https://genius.com/Mac-miller-kick-push-lyrics
https://genius.com/Mac-miller-thanks-for-comin-out-lyrics



https://genius.com/Mac-miller-come-around-lyrics
https://genius.com/Mac-miller-get-mines-lyrics
https://genius.com/Mac-miller-in-love-with-this-bud-lyrics
https://genius.com/Mac-miller-my-biography-lyrics
https://genius.com/Mac-miller-smoke-signals-lyrics
https://genius.com/Mac-miller-take-me-to-paradise-lyrics



https://genius.com/Mac-miller-tambourine-dream-lyrics
https://genius.com/Mac-miller-the-song-that-changed-everything-lyrics
https://genius.com/Mac-miller-do-you-have-a-destination-lyrics
https://genius.com/Mac-miller-mrs-deborah-downer-lyrics
https://genius.com/Mac-miller-stoned-lyrics
https://genius.com/Mac-miller-colors-and-shapes-demo-lyrics
https://genius.com/Mac-miller-excelsior-lyrics
https://genius.com/Mac-miller-friendly-hallucinations-lyrics
https://genius.com/Mac-miller-funny-papers-lyrics
https://genius.com/Mac-miller-manakins-lyrics
https://genius.com/Mac-miller-he-finally-sleeps-lyrics
https://genius.com/Mac-miller-transformations-lyrics
https://genius.com/Mac-miller-uber-demo-lyrics
https://genius.com/Mac-miller-smile-demo-lyrics
https://genius.com/Mac-miller-tomorrow-will-never-know-lyrics
https://genius.com/Mac-miller-bobs-dementia-lyrics



https://genius.com/Mac-miller-superweirdo-freestyle-lyrics
https://genius.com/Mac-miller-nervous-breakdown-lyrics
https://genius.com/Mac-miller-stupid-lyrics
https://genius.com/Mac-miller-empires-lyrics
https://genius.com/Mac-miller-coolin-lyrics
https://genius.com/Mac-miller-cat-and-mouse-lyrics
https://genius.com/Mac-miller-start-a-revolution-lyrics
https://genius.com/Mac-miller-your-shoes-are-untied-lyrics
https://genius.com/Mac-miller-when-i-was-young-lyrics
https://genius.com/Mac-miller-what-made-me-this-way-lyrics
https://genius.com/Mac-miller-what-i-need-head-lyrics
https://genius.com/Mac-miller-weekend-demo-lyrics
https://genius.com/Mac-miller-waterfalls-lyrics
https://genius.com/Mac-miller-watch-the-sun-come-up-lyrics
https://genius.com/Mac-miller-wake-up-2019-lyrics
https://genius.com/Mac-miller-untitled-snippet-4-7-20-lyrics
https://genius.com/Mac-miller-untitled-snippet-2-26-13-lyrics
https://genius.com/Mac-miller-unhappy-lyrics
https://genius.com/Mac-miller-tv-lyrics
https